In [ ]:
%pylab inline
%config InlineBackend.figure_format='retina'
from scipy.integrate import odeint
from scipy.optimize import curve_fit
import pandas as pd

In [ ]:
df_ps02 = pd.read_csv('data/03_PS2.csv')
df_micron = df_ps02['D50_micron']
t_min = df_ps02['Time_min']
df_ps02.head()

In [ ]:
fig, ax = plt.subplots()
ax.plot(t_min,df_micron,'ob', label='data')
ax.set_xscale('log'),
ax.set_xlabel('Time (min)')
ax.set_ylabel('Floc Size ($\mu$m)')
legend(loc=4);

In [ ]:
df = df_micron/10**6
t_sec = t_min*60

In [ ]:
def f(D, t, Ka, Kb):
    """
    ODE to be integrated: dDdt = A-B
    """
    # kind of constant:
    nf = 2.25
    p = 3.-nf
    q = 0.5
    Fy = 1e-10 # (N)
    rhos = 2500.  # (kg/m3)
    mu = 1.002e-3 # dynamic viscosity @ 20 degC (N s m-2)
    
    Dp = 7e-6 # (m)
    G = 35. # (s-1)
    C = 50.e-3 # (kg/m3)
    
    # aggregation and breakup terms
    A = D**(4-nf)*Ka*Dp**(nf-3)*G*C/(nf*rhos)
    B = (D-Dp)**p * D**(2*q+1) *Kb *((Dp**(-p))/nf) *(mu/Fy)**q *G**(q+1)
    dfdt = A-B
    return dfdt

# Ka = 0.73
# Kb = 9.5e-6

# initial values
D0 = 20e-6 # (m)
t = np.linspace(60,100000.,1000)

def D(t, Ka, Kb, D0):
    """
    Solution to the ODE y'(t) = f(t,y,a,b) with initial condition y(0) = y0
    """
    D = odeint(f, D0, t, args=(Ka, Kb))
    return D.ravel()

popt, cov = curve_fit(D, t_sec, df, [0.1,0,D0])
Ka_fit, Kb_fit, D0_fit = popt

In [ ]:
popt

In [ ]:
fig, ax = plt.subplots()
ax.plot(t_min,df_micron,'ob', label='data')
ax.plot(t/60,D(t, Ka_fit, Kb_fit, D0_fit)*10**6,'r', label='fit')
ax.set_xscale('log'),
ax.set_xlabel('Time (min)')
ax.set_ylabel('Floc Size ($\mu$m)')
legend(loc=4);